In [203]:
#!pip install langchain_openai
#!pip install chromadb
#!pip install transformers
#!pip install faiss-cpu
#!pip install -qU langchain-huggingface
#!pip install -U langchain-community
#!pip install langchain
import os


!pip install langsmith
!pip install langchainhub
!pip install openai
!pip install --upgrade --quiet  openllm
!pip install -U langchain_anthropic
!pip install openllm
!pip install huggingface_hub

Python(76685) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(76688) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(76698) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(76699) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Python(76702) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(76709) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(76710) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [204]:
import bs4
from langchain import hub
from langchain.llms import OpenLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pandas as pd
from sentence_transformers import SentenceTransformer

loader = WebBaseLoader(
    web_paths=("https://other-docs.snowflake.com/en/polaris/overview",),
)
docs = loader.load()

In [205]:
data = []
for doc in docs:
  page_number = doc.metadata['source']
  page_title = doc.metadata['title']
  page_content = doc.page_content
  data.append([page_number,page_title, page_content])

df = pd.DataFrame(data, columns=['Page','Page Title','Content'])

In [206]:
import requests
from bs4 import BeautifulSoup

def get_english_links(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        hrefs = {link.get('href') for link in links if link.get('href')}
        english_links = {href for href in hrefs if '/en/' in href}
        english_links = {href if href.startswith('http') else url + href for href in english_links}
        return english_links
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return set()

def load_documents_from_links(links):
    loader = WebBaseLoader(web_paths=tuple(links))
    docs = loader.load()
    data = []
    for doc in docs:
        page_number = doc.metadata['source']
        page_title = doc.metadata['title']
        page_content = doc.page_content
        data.append([page_number, page_title, page_content])
    return pd.DataFrame(data, columns=['Page', 'Page Title', 'Content']), docs


url = "https://docs.snowflake.com"
english_links = get_english_links(url)
if english_links:
    print(f"Found {len(english_links)} English links. Loading documents...")
    df, docs = load_documents_from_links(english_links)
    print(df.head())
else:
    print("No English links found or webpage could not be accessed.")

Found 40 English links. Loading documents...
                                                Page  \
0  https://docs.snowflake.com/en/release-notes/pe...   
1  https://docs.snowflake.com/en/developer-guide/...   
2  https://docs.snowflake.com/en/user-guide/wareh...   
3            https://docs.snowflake.com/en/tutorials   
4  https://docs.snowflake.com/en/user-guide/admin...   

                                          Page Title  \
0  Performance Improvements | Snowflake Documenta...   
1             Snowpark API | Snowflake Documentation   
2       Virtual warehouses | Snowflake Documentation   
3                Tutorials - Snowflake Documentation   
4           Trial accounts | Snowflake Documentation   

                                             Content  
0  Performance Improvements | Snowflake Documenta...  
1  Snowpark API | Snowflake DocumentationDOCUMENT...  
2  Virtual warehouses | Snowflake DocumentationDO...  
3  Tutorials - Snowflake DocumentationDOCUMENTATI...  
4  Tri

In [207]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain import hub
from uuid import uuid4

embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [208]:
documents = vectorstore.similarity_search('What is a warehouse?')

In [209]:
len(documents)

4

In [210]:
for docs in documents:
    print(docs.page_content)
    print('------------------------------------------------------------------------------------------------------------------------------------')

Virtual warehouses | Snowflake DocumentationDOCUMENTATION/Getting StartedGuidesDeveloperReferenceReleasesTutorialsPolaris CatalogStatusOverviewSnowflake HorizonConnecting to SnowflakeVirtual warehousesOverviewMulticlusterConsiderationsWorking with warehousesQuery Acceleration ServiceMonitoring loadSnowpark-optimized warehousesDatabases, Tables, & ViewsData TypesData LoadingData UnloadingQueriesData Sharing and CollaborationSnowflake AI & MLAlerts & NotificationsSecurityData GovernancePrivacyOrganizations & AccountsBusiness Continuity & Data RecoveryPerformance OptimizationCost & BillingGuidesVirtual warehouses

Virtual warehouses¶
A virtual warehouse, often referred to simply as a “warehouse”, is a cluster of compute resources in Snowflake. A virtual warehouse is
available in two types:

Standard
Snowpark-optimized

A warehouse provides the required resources, such as CPU, memory, and temporary storage, to
perform the following operations in a Snowflake session:

Executing SQL SELECT s

In [211]:
retriever.get_relevant_documents('What is the Warehouse?')

[Document(metadata={'source': 'https://docs.snowflake.com/en/user-guide/warehouses', 'title': 'Virtual warehouses | Snowflake Documentation', 'language': 'en'}, page_content='Virtual warehouses | Snowflake DocumentationDOCUMENTATION/Getting StartedGuidesDeveloperReferenceReleasesTutorialsPolaris CatalogStatusOverviewSnowflake HorizonConnecting to SnowflakeVirtual warehousesOverviewMulticlusterConsiderationsWorking with warehousesQuery Acceleration ServiceMonitoring loadSnowpark-optimized warehousesDatabases, Tables, & ViewsData TypesData LoadingData UnloadingQueriesData Sharing and CollaborationSnowflake AI & MLAlerts & NotificationsSecurityData GovernancePrivacyOrganizations & AccountsBusiness Continuity & Data RecoveryPerformance OptimizationCost & BillingGuidesVirtual warehouses\n\nVirtual warehouses¶\nA virtual warehouse, often referred to simply as a “warehouse”, is a cluster of compute resources in Snowflake. A virtual warehouse is\navailable in two types:\n\nStandard\nSnowpark-o

In [212]:
from getpass import getpass
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
import os
#HUGGINGFACEHUB_API_TOKEN = getpass()


#os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


template = """
Answer the question based only on the following context:
{context}

QUESTION: {question}


""".strip()
question = 'What is a virtual Warehouse?'

prompt = PromptTemplate.from_template(template)

print(prompt)

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    #huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)
llm_chain = prompt | llm
print(llm_chain.invoke({"question": question, 'context':retriever}))


input_variables=['context', 'question'] template='Answer the question based only on the following context:\n{context}\n\nQUESTION: {question}'
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/abdulhakeemoyaqoob/.cache/huggingface/token
Login successful


 What library is used for creating it and how to perform a search using it?

A: A virtual Warehouse is not a term used in the context provided. The context mentions FAISS (Facebook AI Similarity Search) and HuggingFaceEmbeddings, which are libraries used for creating and interacting with vector databases. The FAISS library is used to create a vectorstore object, and a search can be performed using the `search` method of the vectorstore object, passing the query vector and search parameters as arguments. For example, to perform a search with k=1, you can use the following code:

```python
results = vectorstore.search(query_vector, search_kwargs)
```

This will return a list containing the vector and index of the closest match in the vectorstore.


In [213]:
question

'What is a virtual Warehouse?'

In [214]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke(question))



ANSWER: A virtual warehouse is a cluster of compute resources in Snowflake. It is available in two types: Standard and Snowpark-optimized. A warehouse provides the required resources, such as CPU, memory, and temporary storage, to perform operations in a Snowflake session, including executing SQL SELECT statements, performing DML operations, loading and unloading data, and more. A warehouse must be running and in use for the session to perform these operations, and while it is running, it consumes Snowflake credits. Warehouses are required for queries and all DML operations. In addition to being defined by its type, a warehouse is also defined by its size and other properties that can be set to help control and automate warehouse activity. Snowpark workloads can be run on both Standard and Snowpark-optimized warehouses, and Snowpark-optimized warehouses are recommended for workloads that have large memory requirements such as ML training use cases. Multi-cluster warehouses enable you